# Project Phase 1 Report
Team 7: Lipsa J., Ji K., Yuanfeng L., Yu L.

## Report format using QQQ

1. Qualitative
* Question, problem, hypothesis, claim, context, motivation
* Definitions, data, methods to be used
* Rationale, assumptions, biases

2. Quantitative:
* Data processing, analysis, visualization
* Documented code and results
* Summary visuals

3. Qualitative:
* Answer, update question/claim, summary, re-contextualization, story, * relate to domain knowledge
* Uncertainty, limitations, caveats
* New problems, next steps

4. Repeat. QQQ-QQQ-QQQ-...
* Break down a large problem into parts
* Alternative approaches to a problem
* Sequence of related pro

## References: 
* Good example of a Jupyter Notebook report: https://nbviewer.jupyter.org/gist/nealcaren/5105037
(https://nbviewer.jupyter.org/gist/nealcaren/5105037)

* QQQ: https://www.bava.stat.vt.edu/wp-content/uploads/2017/08/Developing-a-New-Interdisciplinary-
Computational-Analytics-Undergraduate-Program-A-Qualitative-Quantitative-Qualitative-Approach.pdf
(https://www.bava.stat.vt.edu/wp-content/uploads/2017/08/Developing-a-New-Interdisciplinary-
Computational-Analytics-Undergraduate-Program-A-Qualitative-Quantitative-Qualitative-Approach.pdf)

* Using visuals to support claims: https://www.cbre.com/research-and-reports/Scoring-Tech-Talent-in-North-
America-2018 (https://www.cbre.com/research-and-reports/Scoring-Tech-Talent-in-North-America-2018)

* Typical industry spam: https://www.seagate.com/files/www-content/our-story/trends/files/idc-seagate-
dataage-whitepaper.pdf (https://www.seagate.com/files/www-content/our-story/trends/files/idc-seagate-
dataage-whitepaper.pdf)